In [247]:
# Run this cell
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline
import re
import nltk
import string
# run this in cmd to avoid an error beforehand for this import
# python -m nltk.downloader stopwords 
from nltk.corpus import stopwords

import random
from sklearn.feature_extraction.text import TfidfVectorizer


In [248]:
PUNCT_TO_REMOVE = string.punctuation
STOPWORDS = set(stopwords.words('english'))

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


In [249]:
df = pd.read_csv('train.csv')
df["text"] = df["text"].str.lower()
df["text"] = df["text"].apply(lambda text: remove_urls(text))
df["text"] = df["text"].apply(lambda text: remove_punctuation(text))
df["text"] = df["text"].apply(lambda text: remove_stopwords(text))
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [250]:
def randNoun(lines):
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    tokenized = nltk.word_tokenize(lines)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
    if (len(nouns) == 0): # if no nouns picked up return rand word in text
        return lines[random.randrange(len(lines))]
    return nouns[random.randrange(len(nouns))]

In [251]:
# preprocessing of the dataset
print(len(df["location"]))
words = {} # will hold unique keywords
count = 0
# getting rid of %20 in keyword and getting a list of all the unqiue keywords
for i in df["keyword"]:
    i = str(i).replace("%20", ' ')
    if (i not in words.keys() and i != "nan"):
        words[i] = count
        count+=1

# filling in missing feature values
# missing keywords get filled in with a keywords already in "words" if that keyword shows up in text, otherwise pick a random word from text
# Missing "location" gets filled in with "Earth"
for i,j in df.iterrows():
    if (pd.isna(j["keyword"])):
        for k in words.keys():
            if k in j["text"]:
                df.at[i, "keyword"] = k
            else:
                df.at[i, "keyword"] = randNoun(j["text"])
    if (pd.isna(j["location"])):
        df.at[i, "location"] = "Earth"
# for i, j in df.iterrows():
#     if (pd.isna(j["location"])):
#         df.drop(i, axis=0, inplace=True)

print(len(df["text"]))


7613
7613


In [252]:
# df.reset_index(inplace=True, drop=True)
# df.head()


In [259]:
# we are going to feature transform 
corpus = df["text"].to_list() # combine all text to a bag of words
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray() # TF-IDF text encoding

#doing hot and cold encoding for keywords
words = {} # will hold unique keywords with a corresponding index
count = 0
for i in df["keyword"]:
    if (i not in words.keys()):
        words[i] = count
        count+=1
print(words)
# keywords_hot_cold = np.zeros((len(df["keyword"]), len(words.keys()))) # init feature shape
# for i in range(keywords_hot_cold.shape[0]):
#     row = np.zeros(keywords_hot_cold.shape[1])
#     row[words[df.at[i, "keyword"]]] = 1 # using the index found in the dict, index the row and set the corresponding value of the keyword to 1
#     keywords_hot_cold[i] = row

keywords_vector =  np.zeros((len(df["keyword"]), 1)) # column vector to store possible values
for i in range(keywords_vector.shape[0]):
    keywords_vector[i] = words[df.at[i, "keyword"]] # find corresponding value of that keyword in the "words" dict and assign that value 
print(keywords_vector)

#doing hot and cold encoding for locations
locations = {}
count = 0
for i in df["location"]:
    if (i not in locations.keys()):
        locations[i] = count
        count+=1
# location_hot_cold = np.zeros((len(df["location"]), len(locations.keys())))
# for i in range(location_hot_cold.shape[0]):
#     row = np.zeros(location_hot_cold.shape[1])
#     row[locations[df.at[i, "location"]]] = 1
#     location_hot_cold[i] = row

locations_vector =  np.zeros((len(df["location"]), 1)) # column vector to store possible values
for i in range(locations_vector.shape[0]):
    locations_vector[i] = words[df.at[i, "location"]] # find corresponding value of that keyword in the "words" dict and assign that value 
print(locations_vector)



{'deeds': 0, 'canada': 1, 'orders': 2, 'wildfires': 3, 'alaska': 4, 'fire': 5, 'flood': 6, 'emergency': 7, 'im': 8, 'heat': 9, 'south': 10, 'count': 11, 'damage': 12, 'whats': 13, 'fruits': 14, 'summer': 15, 'car': 16, 'goooooooaaaaaal': 17, 'i': 18, 'cool': 19, 'love': 20, 'day': 21, 'looooool': 22, 'wayi': 23, 'week': 24, 'girlfriend': 25, 'cooool': 26, 'pasta': 27, 'end': 28, 'ablaze': 29, 'accident': 30, 'aftershock': 31, 'airplane%20accident': 32, 'ambulance': 33, 'annihilated': 34, 'annihilation': 35, 'apocalypse': 36, 'armageddon': 37, 'army': 38, 'arson': 39, 'arsonist': 40, 'attack': 41, 'attacked': 42, 'avalanche': 43, 'battle': 44, 'bioterror': 45, 'bioterrorism': 46, 'blaze': 47, 'blazing': 48, 'bleeding': 49, 'blew%20up': 50, 'blight': 51, 'blizzard': 52, 'blood': 53, 'bloody': 54, 'blown%20up': 55, 'body%20bag': 56, 'body%20bagging': 57, 'body%20bags': 58, 'bomb': 59, 'bombed': 60, 'bombing': 61, 'bridge%20collapse': 62, 'buildings%20burning': 63, 'buildings%20on%20fire':

KeyError: 'Earth'

In [257]:
print(keywords_hot_cold.shape)
print(location_hot_cold.shape)


(7613, 267)
(7613, 3341)


In [254]:
# combine all features and split into training and validation sets
X = np.hstack((np.ones((X.shape[0], 1)), keywords_hot_cold, location_hot_cold, X))
X_train, X_val,  y_train, y_val = train_test_split(X, df["target"], test_size=.4)
# poly = PolynomialFeatures(1)
# X_train = poly.fit_transform(X_train)
# X_val = poly.fit_transform(X_val)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(4567, 21323) (4567,) (3046, 21323) (3046,)


In [255]:
def poly_regression(X,y,lambda1): # We are using X for the design matrix.  It might be that X is in Z-space
    d = X.shape[1]
    N = X.shape[0]
    I_prime = np.identity(d)
    I_prime[0,0] = 0
    w = np.linalg.inv(X.T.dot(X)+(N*lambda1)*I_prime).dot(X.T.dot(y))

    return w

In [256]:
w = poly_regression(X_train, y_train, .001)
print(w)

KeyboardInterrupt: 

In [ ]:
yhat_val = X_val.dot(w)
E_val = (1/yhat_val.shape[0])*(np.sum((yhat_val - y_val)**2))
print(yhat_val.shape[0])
print(E_val)

3046
0.17477143550920526
